# 1. Importing the required libraries

In [1]:
!pip install nltk
!pip install pgeocode
!pip install -U spacy
!python -m spacy download en_core_web_lg
!pip install spacy[transformers]
import nltk
nltk.download('stopwords')

     |████████████████████████████████| 12.8MB 235kB/s 
     |████████████████████████████████| 460kB 46.2MB/s 
     |████████████████████████████████| 624kB 44.9MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 9.1MB 35.8MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
2021-07-06 17:49:35.014953: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8MB 24kB/s 
✔ Download and installation successful
You c

True

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pgeocode
import plotly.express as px
import spacy
import en_core_web_lg
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', 1000)
pd.set_option('display.float_format', str)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading spacy model for furthur use

In [4]:
nlp = en_core_web_lg.load()

# 2. Reading the Datasets

`Banklist.csv` Data source :
  - https://www.finder.com.au/list-of-banks-in-australia
  - https://en.wikipedia.org/wiki/List_of_banks_in_Australia

In [5]:
df = pd.read_csv("/content/drive/MyDrive/ANZ/data.csv")
banklistdf = pd.read_csv("/content/drive/MyDrive/ANZ/banklist.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,1,2,3,4,5,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,42) have mixed types.Specify dtype option on import or set low_memory=False.



In [6]:
df.head(3)

,Agency Name,Parent Contract ID,Contract ID,Publish Date,Start Date,End Date,Value,Amendment Date,Amendment Start Date,Amendments Value,Description,Agency Ref ID,UNSPSC Code,UNSPSC Title,Procurement Method,ATM ID,SON ID,Panel Arrangement,Confidentiality Contract Flag,Confidentiality Contract Reason,Confidentiality Outputs Flag,Confidentiality Outputs Reason,Consultancy Flag,Consultancy Reason,Amendment Reason,Supplier Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country,Supplier ABN Exempt,Supplier ABN,Contact Name,Contact Phone,Branch,Division,Office Postcode,Applicable Publish Date,Applicable FY Year,Applicable Value,Duration Years,Applicable Start Date
0,Department of Defence,NaN,CN3620990,20/8/19,1/5/19,31/5/19,16163.4,NaN,NaN,nan,Fitness Equipment,4501071435,49200000.0,Fitness equipment,Limited tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,BLP TRAINING & SERVICES PTY LTD,NaN,BEENLEIGH,4207,QLD,AUSTRALIA,No,40079623340.0,Army Forces Command,0,Army Forces Command,ARMY HEADQUARTERS,2021,20/8/19,2020.0,16163.0,0.0,1/5/19
1,Australian Bureau of Statistics,CN3602517,CN3602517-A1,21/6/19,1/7/18,30/6/23,1964000.0,25/5/20,25/5/20,648000.0,Software Maintenance,ABS2018.450,81112200.0,Software maintenance and support,Open tender,NaN,SON3520963,Yes,No,NaN,No,NaN,No,NaN,Variation,IBM Australia Limited,NaN,Pennant Hills,1715,NSW,AUSTRALIA,No,79000024733.0,"Director, procurement",NaN,NaN,NaN,2617,25/5/20,2020.0,648000.0,3.0,25/5/20
2,Services Australia,NaN,CN3671573,7/4/20,19/3/20,27/3/20,155997.6,NaN,NaN,nan,Training Services,D296030217,86000000.0,Education and Training Services,Open tender,NaN,SON3393682,Yes,No,NaN,No,NaN,No,NaN,NaN,Benestar Group Pty Ltd,PO Box Q1436,QVB Post Office Sydney,1230,NSW,Australia,No,61003536472.0,PURCHASING HELPDESK,1800 906 144,DHS Branch,DHS Division,2900,7/4/20,2020.0,155998.0,0.0,19/3/20


In [7]:
banklistdf.head(3)

,Bank,Headquarters,Established,Owned by / Subsidiary of
0,86400 Limited,Sydney,2019,NaN
1,AMP Bank,Sydney,1849,NaN
2,Australia & New Zealand Banking Group (ANZ),Melbourne,1835,NaN


# 3. Exploratory Data Analysis

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292903 entries, 0 to 292902
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Agency Name                      81174 non-null  object 
 1   Parent Contract ID               15920 non-null  object 
 2   Contract ID                      81174 non-null  object 
 3   Publish Date                     81174 non-null  object 
 4   Start Date                       81174 non-null  object 
 5   End Date                         81174 non-null  object 
 6   Value                            81174 non-null  float64
 7   Amendment Date                   15920 non-null  object 
 8   Amendment Start Date             15990 non-null  object 
 9   Amendments Value                 15928 non-null  float64
 10  Description                      81174 non-null  object 
 11  Agency Ref ID                    79716 non-null  object 
 12  UNSPSC Code     

In [9]:
banklistdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Bank                      228 non-null    object
 1   Headquarters              119 non-null    object
 2   Established               72 non-null     object
 3   Owned by / Subsidiary of  10 non-null     object
dtypes: object(4)
memory usage: 7.2+ KB


In [10]:
df.describe()

,Value,Amendments Value,UNSPSC Code,Supplier ABN,Applicable FY Year,Applicable Value,Duration Years
count,81174.0,15928.0,81174.0,77510.0,81174.0,81174.0,81174.0
mean,4899027.464109137,1303384.4963033623,67698995.80524552,53873333191.25513,2020.0,664935.0670288516,0.727067780323749
std,109827357.63649951,15509753.691141438,21409666.996033147,24519615207.061092,0.0,9665913.02006304,1.534367186776721
min,10000.0,0.0,10100000.0,11000013098.0,2020.0,0.0,0.0
25%,27000.0,37025.212499999994,46161600.0,33071052287.0,2020.0,23796.0,0.0
50%,77472.23000000001,109999.125,80111600.0,52119884945.0,2020.0,60227.0,1.0
75%,294370.1225,312400.0,81100000.0,73446832485.75,2020.0,191755.0,1.0
max,14012651773.0,818246874.2,94130000.0,99980701814.0,2020.0,818246874.0,182.0


In [11]:
banklistdf.describe()

,Bank,Headquarters,Established,Owned by / Subsidiary of
count,228,119,72,10
unique,210,21,60,7
top,Bank of Melbourne,Sydney,1959,Subsidiary of Westpac
freq,2,48,3,3


# 4. Data Preprocessing

In [12]:
dfNotNa = df.loc[df["Agency Name"].notna(), :]
dfNotNa["Parent Contract ID"].fillna(dfNotNa["Contract ID"], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
for col in dfNotNa:
    print(col +": "+ str(len(df[col].unique())))

Agency Name: 120
Parent Contract ID: 12058
Contract ID: 81175
Publish Date: 1353
Start Date: 1893
End Date: 2035
Value: 55223
Amendment Date: 267
Amendment Start Date: 627
Amendments Value: 11709
Description: 32123
Agency Ref ID: 70751
UNSPSC Code: 526
UNSPSC Title: 526
Procurement Method: 4
ATM ID: 7282
SON ID: 683
Panel Arrangement: 3
Confidentiality Contract Flag: 3
Confidentiality Contract Reason: 30
Confidentiality Outputs Flag: 3
Confidentiality Outputs Reason: 22
Consultancy Flag: 3
Consultancy Reason: 4
Amendment Reason: 5237
Supplier Name: 19617
Supplier Address: 8743
Supplier Suburb: 5644
Supplier Postcode: 2805
Supplier State: 10
Supplier Country: 126
Supplier ABN Exempt: 3
Supplier ABN: 11335
Contact Name: 1785
Contact Phone: 898
Branch: 624
Division: 481
Office Postcode: 149
Applicable Publish Date: 279
Applicable FY Year: 2
Applicable Value: 45409
Duration Years: 36
Applicable Start Date: 1183


In [14]:
dfNotNa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81174 entries, 0 to 81173
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Agency Name                      81174 non-null  object 
 1   Parent Contract ID               81174 non-null  object 
 2   Contract ID                      81174 non-null  object 
 3   Publish Date                     81174 non-null  object 
 4   Start Date                       81174 non-null  object 
 5   End Date                         81174 non-null  object 
 6   Value                            81174 non-null  float64
 7   Amendment Date                   15920 non-null  object 
 8   Amendment Start Date             15990 non-null  object 
 9   Amendments Value                 15928 non-null  float64
 10  Description                      81174 non-null  object 
 11  Agency Ref ID                    79716 non-null  object 
 12  UNSPSC Code       

In [15]:
bankcreditlist = list(banklistdf.loc[~banklistdf["Bank"].str.lower().str.contains("bank|credit") , "Bank"]
     .str.replace(" Limited", "")
     .str.replace(" Ltd", "")
     .str.replace('\(.*$', "", regex=True)
     .str.replace('\[.*$', "", regex=True))

In [16]:
words = re.findall(r'\w+', " ".join(list(dfNotNa["UNSPSC Title"].unique())).lower())
newwordlist = [word for word in words if not word in stopwords.words()]
Counter(newwordlist)

Counter({'access': 1,
         'accessories': 25,
         'accommodation': 1,
         'accounting': 1,
         'activities': 1,
         'additives': 2,
         'adhesives': 1,
         'administration': 4,
         'administrative': 2,
         'administrators': 1,
         'advertising': 5,
         'advisory': 2,
         'aeronautical': 1,
         'aerospace': 1,
         'agencies': 1,
         'agents': 1,
         'agricultural': 1,
         'aid': 2,
         'aids': 2,
         'air': 5,
         'aircraft': 10,
         'airfield': 1,
         'airport': 2,
         'alarm': 1,
         'alloys': 1,
         'ammunition': 2,
         'anaesthesia': 1,
         'analysis': 2,
         'animal': 2,
         'animals': 1,
         'anti': 1,
         'apparel': 1,
         'appearance': 1,
         'appliances': 1,
         'application': 1,
         'aquaculture': 2,
         'architectural': 2,
         'archive': 1,
         'arms': 1,
         'art': 2,
         'artist

Identifying similar words to bank in `UNSPSC Title` column using spacy NLP

In [17]:
similaritydioct = {}
for eachword in newwordlist:
  similaritydioct[eachword] = nlp(eachword).similarity(nlp("bank"))
similaritydioct = sorted(similaritydioct.items(), key=lambda x: x[1], reverse=True)
similaritydioct

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning:

[W008] Evaluating Doc.similarity based on empty vectors.



[('banking', 0.776921766945123),
 ('financial', 0.6691584119843744),
 ('credit', 0.6683419834038032),
 ('finance', 0.6244395322569233),
 ('debt', 0.6106305690336367),
 ('investment', 0.5796768036950611),
 ('financing', 0.5496427924644919),
 ('tax', 0.4894074516624606),
 ('card', 0.4837748461766773),
 ('insurance', 0.47551415457996005),
 ('institutions', 0.4635415016588086),
 ('accounting', 0.4579844721965832),
 ('estate', 0.4569045883845878),
 ('corporate', 0.45567144035065865),
 ('business', 0.4431012911318381),
 ('fees', 0.4376809552826809),
 ('market', 0.41568868558774674),
 ('economic', 0.40442313322146867),
 ('security', 0.4043766493452597),
 ('office', 0.4019643310152811),
 ('personal', 0.3992797335512578),
 ('transfer', 0.39614988100962556),
 ('postal', 0.39548325413161856),
 ('consumer', 0.393906539237221),
 ('management', 0.39279456086861486),
 ('real', 0.3903542716834028),
 ('property', 0.3900866098988434),
 ('audit', 0.3858480520240245),
 ('risk', 0.38514951760080746),
 ('st

getting the string for matching top 10 keywoprds in domain of banking

In [18]:
commonWordStringInTitle = "|".join([i[0].lower() for i in similaritydioct[:10]])

Getting list of suppliers that have term `bank` or is `ANZ` 

In [19]:
for i in dfNotNa["Supplier Name"].unique():
  if "Bank".lower() in str(i).lower():
    print(i)
  elif "ANZ".lower() in str(i).lower():
    print(i)

PEOPLEBANK AUSTRALIA LTD
Manzar Azari
2377MIZUHOTRUSTBANKREALESTATE
SERVICIOS CENTRALES DE COBRANZA
Content Bank Australia Pty Ltd
Travelodge Southbank
Peoplebank Australia Ltd
Peoplebank Australia Pty Ltd
PeopleBank Australia Limited
Tanzanian Veterinary Laboratory Agency
PeopleBank Australia Pty Ltd
ECA INTERNATIONAL (Aust Bank Details)
Elizabeth Anne Cheeseman Deutsche Bank Place
Lawyerbank Pty Ltd
Commonwealth Bank
Maddocks - Trust Bank Account
Rubik3 Pty Ltd (DO NOT USE UNTIL BANK DETAILS UPDATED)
Rydges South Bank Brisbane
PEOPLEBANK AUSTRALIA LIMITED
Australia and New Zealand Banking Group Limited
Deutsche Bank AG Australia
Commonwealth Bank of Australia
Reserve Bank of Australia
ANZ
Citibank Limited
RESERVE BANK OF AUSTRALIA
INTERNATIONAL ASSOCIATION OF DEPOSIT INSURERS C/O BANK FOR INTERMANTIONAL SETTLEMENTS
INVESTMENT PROPERTY DATABANK PTY LTD
DFC LEGAL PTY LTD T/A LAWYERBANK
WOMEN IN BANKING AND FINANCE
TECH DATA ADVANCED SOLUTIONS (ANZ) LIMITED
Peoplebank Australia
Peopleba

In [20]:
for eachcomp in [i.lower() for i in bankcreditlist]:
  if dfNotNa.loc[dfNotNa["Supplier Name"].str.lower().str.contains(eachcomp) , :].shape[0] > 0:
     print(eachcomp)


clean energy finance corporation
up
ecu
imb
catalyst
asl
the mac


In [21]:
dfNotNa.loc[dfNotNa["Supplier Name"].str.lower().str.contains("the mac") , :]

,Agency Name,Parent Contract ID,Contract ID,Publish Date,Start Date,End Date,Value,Amendment Date,Amendment Start Date,Amendments Value,Description,Agency Ref ID,UNSPSC Code,UNSPSC Title,Procurement Method,ATM ID,SON ID,Panel Arrangement,Confidentiality Contract Flag,Confidentiality Contract Reason,Confidentiality Outputs Flag,Confidentiality Outputs Reason,Consultancy Flag,Consultancy Reason,Amendment Reason,Supplier Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country,Supplier ABN Exempt,Supplier ABN,Contact Name,Contact Phone,Branch,Division,Office Postcode,Applicable Publish Date,Applicable FY Year,Applicable Value,Duration Years,Applicable Start Date
58089,Department of Health,CN3631377,CN3631377,1/10/19,27/9/19,30/6/20,850800.06,NaN,NaN,nan,Procure Research Data,4500134773,93131700.0,Health programs,Limited tender,HEALTH/2019-20,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,The Macfarlane Burnet Institute for Medical Re...,NaN,MELBOURNE,3004,VIC,Australia,No,49007349984.0,BMU,02 6289 1555,NaN,Pharmaceutical Benefits Division,2606,1/10/19,2020.0,850800.0,1.0,27/9/19


## Q1. Which customers should Bank X provide their service to and why? Provide the criteria, explanation or approach to obtain the list of customers.

In [22]:
finaldata = dfNotNa.loc[(dfNotNa["Supplier Name"].str.lower().str.contains("bank|credit|clean energy finance corporation| up |the mac")) |
                        (dfNotNa["UNSPSC Title"].str.lower().str.contains(commonWordStringInTitle)), [ "Agency Name","Value", "Duration Years"]].groupby("Agency Name").agg(['mean',
                                                                                                                                                                             'median',
                                                                                                                                                                             'sum',
                                                                                                                                                                             'min',
                                                                                                                                                                             'max',
                                                                                                                                                                             'count'])

In [23]:
finaldata = finaldata.loc[finaldata['Value',  'count'] > 1, :]
finaldata.sort_values(by=('Value','mean'),ascending=False)


Value  \
                                                                 mean   
Agency Name                                                             
Department of Defence                               62115961.60157575   
Regional Investment Corporation                          10415038.335   
Australian Taxation Office                          2516865.112714287   
Australian Office of Financial Management           2379462.538421053   
Department of Foreign Affairs and Trade - Austr... 1916613.0821428571   
...                                                               ...   
Attorney-General's Department                      23542.445714285714   
Department of Employment, Skills, Small and Fam...            19415.0   
Australian Commission for Law Enforcement Integ...           16408.57   
Royal Australian Mint                                        12233.65   
Inspector-General of Taxation                      11552.404999999999   

                                                                       \
                                                               median   
Agency Name                                                             
Department of Defence                                      2961878.84   
Regional Investment Corporation                          10415038.335   
Australian Taxation Office                                  307690.24   
Australian Office of Financial Management                   3217500.0   
Department of Foreign Affairs and Trade - Austr...           108006.6   
...                                                               ...   
Attorney-General's Department                                 14212.0   
Department of Employment, Skills, Small and Fam...            17204.0   
Australian Commission for Law Enforcement Integ...           16408.57   
Royal Australian Mint                                        12233.65   
Inspector-General of Taxation                      11552.404999999999   

                                                                       \
                                                                  sum   
Agency Name                                                             
Department of Defence                              10249133664.259998   
Regional Investment Corporation                           20830076.67   
Australian Taxation Office                         352361115.78000015   
Australian Office of Financial Management          45209788.230000004   
Department of Foreign Affairs and Trade - Austr...        26832583.15   
...                                                               ...   
Attorney-General's Department                               4119928.0   
Department of Employment, Skills, Small and Fam...            77660.0   
Australian Commission for Law Enforcement Integ...           32817.14   
Royal Australian Mint                                         24467.3   
Inspector-General of Taxation                      23104.809999999998   

                                                                          \
                                                        min          max   
Agency Name                                                                
Department of Defence                              11013.81 3011828390.0   
Regional Investment Corporation                    103950.0  20726126.67   
Australian Taxation Office                          11000.0  206000311.0   
Australian Office of Financial Management           55000.0    4180000.0   
Department of Foreign Affairs and Trade - Austr...  11000.0   23650000.0   
...                                                     ...          ...   
Attorney-General's Department                       10098.0     500000.0   
Department of Employment, Skills, Small and Fam...  10098.0      33154.0   
Australian Commission for Law Enforcement Integ...  12075.3     20741.84   
Royal Australian Mint                               10717.3      13750.0   
Inspect

In [24]:
finaldata.sort_values(by=('Value','max'),ascending=False)

Value  \
                                                                 mean   
Agency Name                                                             
Department of Defence                               62115961.60157575   
Australian Taxation Office                          2516865.112714287   
Department of Finance                               616434.0464893618   
Department of Education, Skills and Employment     1606282.8495833334   
Department of Foreign Affairs and Trade - Austr... 1916613.0821428571   
...                                                               ...   
National Capital Authority                                    36000.0   
Department of Employment, Skills, Small and Fam...            19415.0   
Australian Commission for Law Enforcement Integ...           16408.57   
Royal Australian Mint                                        12233.65   
Inspector-General of Taxation                      11552.404999999999   

                                                                       \
                                                               median   
Agency Name                                                             
Department of Defence                                      2961878.84   
Australian Taxation Office                                  307690.24   
Department of Finance                                         41258.5   
Department of Education, Skills and Employment               291544.0   
Department of Foreign Affairs and Trade - Austr...           108006.6   
...                                                               ...   
National Capital Authority                                    36000.0   
Department of Employment, Skills, Small and Fam...            17204.0   
Australian Commission for Law Enforcement Integ...           16408.57   
Royal Australian Mint                                        12233.65   
Inspector-General of Taxation                      11552.404999999999   

                                                                       \
                                                                  sum   
Agency Name                                                             
Department of Defence                              10249133664.259998   
Australian Taxation Office                         352361115.78000015   
Department of Finance                              57944800.370000005   
Department of Education, Skills and Employment            38550788.39   
Department of Foreign Affairs and Trade - Austr...        26832583.15   
...                                                               ...   
National Capital Authority                                    72000.0   
Department of Employment, Skills, Small and Fam...            77660.0   
Australian Commission for Law Enforcement Integ...           32817.14   
Royal Australian Mint                                         24467.3   
Inspector-General of Taxation                      23104.809999999998   

                                                                          \
                                                        min          max   
Agency Name                                                                
Department of Defence                              11013.81 3011828390.0   
Australian Taxation Office                          11000.0  206000311.0   
Department of Finance                               10000.0  36905529.63   
Department of Education, Skills and Employment      26325.0   25413697.0   
Department of Foreign Affairs and Trade - Austr...  11000.0   23650000.0   
...                                                     ...          ...   
National Capital Authority                          36000.0      36000.0   
Department of Employment, Skills, Small and Fam...  10098.0      33154.0   
Australian Commission for Law Enforcement Integ...  12075.3     20741.84   
Royal Australian Mint                               10717.3      13750.0   
Inspect

In [25]:
dfNotNa.loc[dfNotNa["Agency Name"] == "Department of Defence",["Agency Name", "Supplier Address", 
         "Supplier Suburb", 
         "Supplier Postcode" , 
         "Supplier State",
         "Supplier Country"]]

,Agency Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country
0,Department of Defence,NaN,BEENLEIGH,4207,QLD,AUSTRALIA
18539,Department of Defence,NaN,JOHNSTON,831,NT,AUSTRALIA
18540,Department of Defence,NaN,LANE COVE NORTH,2066,NSW,AUSTRALIA
18541,Department of Defence,NaN,THEBARTON,5031,SA,AUSTRALIA
18542,Department of Defence,NaN,LANE COVE WEST,2066,NSW,AUSTRALIA
...,...,...,...,...,...,...
51357,Department of Defence,NaN,ALBURY,2640,NSW,AUSTRALIA
51358,Department of Defence,NaN,NORTH SYDNEY,2060,NSW,AUSTRALIA
51359,Department of Defence,NaN,GRIFFITH,2603,ACT,AUSTRALIA
51360,Department of Defence,NaN,ARCHERFIELD,4108,QLD,AUSTRALIA


## Q2. To meet with agency/supplier representatives regularly, where should Bank X set up their office(s) to serve them better?

Getting the list of unique post code from the dataset in Australia

In [26]:
postCodeList = []
for eachCustomer in list(finaldata.sort_values(by=('Value','max'),ascending=False).index):
  postCodeList.append(dfNotNa.loc[(dfNotNa["Agency Name"] == eachCustomer) & 
              (dfNotNa["Supplier Country"] == "AUSTRALIA"),"Supplier Postcode"].values)
flat_list = [item for sublist in postCodeList for item in sublist]

Getting longitude and lattitude for each post code in Australia 

In [27]:
postcodelistlonglat = []
nomi = pgeocode.Nominatim('AU')
for eachpost in set(flat_list):
  postcodelistlonglat.append(nomi.query_postal_code(eachpost))

Converting the information to dataframe

In [28]:
longlatdf = pd.DataFrame(postcodelistlonglat)
longlatdf.head()

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,3340,AU,"Balliang, Maddingley, Rowsley, Parwan, Darley,...",Victoria,VIC,MELB NORTH WEST,nan,nan,nan,-37.8573,144.42080769230768,4.0
0,6845,AU,Perth,Western Australia,WA,CURTIN,nan,nan,nan,-31.9522,115.8614,4.0
0,6910,AU,Claremont,Western Australia,WA,CURTIN,nan,nan,nan,-31.9796,115.7823,4.0
0,4567,AU,"Sunrise Beach, Sunshine Beach, Noosa Heads, Ca...",Queensland,QLD,SUNSHINE MT ISA,nan,nan,nan,-26.4157,153.10132499999997,4.0
0,3156,AU,"Upper Ferntree Gully, Lysterfield, Lysterfield...",Victoria,VIC,FERNTREE GULLY,nan,nan,nan,-37.8963,145.28789999999998,4.0


Plotting an static plot

In [29]:
fig = px.scatter_geo(longlatdf,lat='latitude',lon='longitude', hover_name="place_name")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

Plotting an interactive plot

In [30]:
px.set_mapbox_access_token("pk.eyJ1IjoicmFzaGJpcmtvaGxpIiwiYSI6ImNrcWs4MnU1djAzZHIzMHRndnFodTRvNjgifQ.r2Wo44z-LyC8PO14JjaeXg")
fig = px.scatter_mapbox(longlatdf,lat='latitude',lon='longitude', hover_name="place_name",
                        zoom=1)
fig.show()

## Q3. Bank X is planning to use machine learning to drive decision-making. Provide a machine learning use case for this dataset with the following details.

Sample implementation of the machine learning algorithm

In [149]:
dataset = dfNotNa.loc[dfNotNa["Agency Name"] == "Department of Defence", :]
dataset["Publish Date"] = pd.to_datetime(dataset["Publish Date"])
dataset = dataset.sort_values(by="Publish Date")
dataset_train = dataset.loc[:dataset.shape[0]/0.80, :]
training_set = dataset_train.iloc[:, 6:7].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [150]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
X_train = []
y_train = []
for i in range(60, training_set.shape[0]):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True,
                   input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.01))

regressor.add(LSTM(units = 50, return_sequences = True, ))
regressor.add(Dropout(0.01))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.01))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.01))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.01))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 300, batch_size = 32)

Epoch 1/300
555/555 [==============================] - 15s 15ms/step - loss: 3.8726e-04
Epoch 2/300
555/555 [==============================] - 8s 15ms/step - loss: 3.0749e-04
Epoch 3/300
555/555 [==============================] - 8s 15ms/step - loss: 2.2990e-04
Epoch 4/300
555/555 [==============================] - 8s 15ms/step - loss: 1.3836e-04
Epoch 5/300
555/555 [==============================] - 8s 15ms/step - loss: 2.4711e-04
Epoch 6/300
555/555 [==============================] - 8s 15ms/step - loss: 2.0116e-04
Epoch 7/300
555/555 [==============================] - 8s 15ms/step - loss: 1.3861e-04
Epoch 8/300
555/555 [==============================] - 8s 15ms/step - loss: 1.7773e-04
Epoch 9/300
555/555 [==============================] - 8s 15ms/step - loss: 8.9499e-05
Epoch 10/300
555/555 [==============================] - 8s 15ms/step - loss: 1.2966e-04
Epoch 11/300
555/555 [==============================] - 8s 15ms/step - loss: 1.0896e-04
Epoch 12/300
555/555 [==================

In [ ]:
regressor.save("/content/drive/MyDrive/ANZ/0.01_model")

In [ ]:
dataset_test = dataset.loc[dataset.shape[0]/0.80:,:]
real_value = dataset_test.iloc[:, 6:7].values

In [ ]:
dataset_total = pd.concat((dataset_train['Value'], dataset_test['Value']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, dataset_test.shape[0]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_value = regressor.predict(X_test)
predicted_value = sc.inverse_transform(predicted_value)

In [ ]:
plt.plot(real_value, color = 'black', label = 'Real Value')
plt.plot(predicted_value, color = 'green', label = 'Predicted Value')
plt.title('Customer Value prediction')
plt.xlabel('Time')
plt.ylabel('Customer Value price')
plt.legend()
plt.show()

In [144]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(real_value[60:], predicted_value)

6084300.900512193

In [145]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(real_value[60:], predicted_value))

11651938.653601967